In [37]:
import requests
import base64
import pandas as pd

In [39]:
# Replace these with your actual Client ID and Secret
client_id = "dd26b37fa50847ee91f2e68dfb5da42a"
client_secret = "286f776767f94a6fb7486a7f9e1819b4"

In [41]:
# Encode Client ID and Secret in Base64
credentials = f"{client_id}:{client_secret}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# Make the POST request
url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {encoded_credentials}"
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(url, headers=headers, data=data)
if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.json())

Access Token: BQB66ecgM06nKf032t2_IOOYdg4o_MJnqStFRvrUWkFQ3f2iKJDRDmMN7Qqfuwdf15qWUs6KxwGEw0BhvhLrIveoAr8oYcBLcIiiPqwECTGbo6jMVYM


In [43]:
# Replace this with your access token from the previous step
access_token = "BQB66ecgM06nKf032t2_IOOYdg4o_MJnqStFRvrUWkFQ3f2iKJDRDmMN7Qqfuwdf15qWUs6KxwGEw0BhvhLrIveoAr8oYcBLcIiiPqwECTGbo6jMVYM"

# Function to search for a song and get album cover URL
def get_album_cover(track_name, artist_name=None):
    # Set up the search query
    query = f"track:{track_name}"
    if artist_name:
        query += f" artist:{artist_name}"
    
    # Search API URL and headers
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "q": query,
        "type": "track",
        "limit": 1  # Only fetch the top result
    }
    
    # Make the request
    response = requests.get(url, headers=headers, params=params)
    
    # Handle the response
    if response.status_code == 200:
        results = response.json()
        if results["tracks"]["items"]:
            # Get the first track's album cover URL
            track = results["tracks"]["items"][0]
            album_cover_url = track["album"]["images"][0]["url"]
            print(f"Album Cover URL for '{track_name}': {album_cover_url}")
            return album_cover_url
        else:
            print(f"No results found for '{track_name}'.")
            return None
    else:
        print("Error:", response.json())
        return None



In [45]:
# Example usage
get_album_cover("Shape of You", "Ed Sheeran")
get_album_cover("Blinding Lights", "The Weeknd")


Album Cover URL for 'Shape of You': https://i.scdn.co/image/ab67616d0000b273ba5db46f4b838ef6027e6f96
Album Cover URL for 'Blinding Lights': https://i.scdn.co/image/ab67616d0000b2738863bc11d2aa12b54f5aeb36


'https://i.scdn.co/image/ab67616d0000b2738863bc11d2aa12b54f5aeb36'

In [47]:
# Example: Adding album cover URLs to the dataset
def add_album_covers_to_dataset(dataset):
    album_covers = []
    for _, row in dataset.iterrows():
        track_name = row["track_name"]
        artist_name = row["artist(s)_name"]
        cover_url = get_album_cover(track_name, artist_name)
        album_covers.append(cover_url)
    
    # Add a new column with album cover URLs
    dataset["album_cover_url"] = album_covers
    return dataset



In [49]:
#Load spotify data
# Load the file with the latin1 encoding
file_path = r"C:\Users\kaybr\Downloads\archive (5)\spotify-2023.csv"
spotify_data = pd.read_csv(file_path, encoding="latin1")

# Display the first few rows to confirm it loaded correctly
# print(spotify_data.head())

# # Add album covers to your dataset
spotify_data_with_covers = add_album_covers_to_dataset(spotify_data)

try:
    spotify_data_with_covers.to_csv("C:/Users/kaybr/Downloads/spotify_with_album_covers_debug.csv", index=False)
    print("File saved successfully!")
except Exception as e:
    print(f"Error saving file: {e}")


Album Cover URL for 'Seven (feat. Latto) (Explicit Ver.)': https://i.scdn.co/image/ab67616d0000b273741fd4807f442af3f7359316
Album Cover URL for 'LALA': https://i.scdn.co/image/ab67616d0000b2730656d5ce813ca3cc4b677e05
Album Cover URL for 'vampire': https://i.scdn.co/image/ab67616d0000b273e85259a1cae29a8d91f2093d
Album Cover URL for 'Cruel Summer': https://i.scdn.co/image/ab67616d0000b273e787cffec20aa2a396a61647
Album Cover URL for 'WHERE SHE GOES': https://i.scdn.co/image/ab67616d0000b273ab5c9cd818ad6ed3e9b79cd1
Album Cover URL for 'Sprinter': https://i.scdn.co/image/ab67616d0000b273e3a09a9ae3f1fa102c110e60
Album Cover URL for 'Ella Baila Sola': https://i.scdn.co/image/ab67616d0000b2732071a0c79802d9375a53bfef
Album Cover URL for 'Columbia': https://i.scdn.co/image/ab67616d0000b2730483b3a4d787ce510915fe3b
Album Cover URL for 'fukumean': https://i.scdn.co/image/ab67616d0000b273017d5e26552345c4b1575b6c
Album Cover URL for 'La Bebe - Remix': https://i.scdn.co/image/ab67616d0000b273a04be3ad7

In [51]:
missing_covers = spotify_data_with_covers[spotify_data_with_covers["album_cover_url"].isna()]
print("Tracks without album covers:", missing_covers)

Tracks without album covers:                                             track_name  \
21   I Can See You (Taylorï¿½ï¿½ï¿½s Version) (From...   
26                       Calm Down (with Selena Gomez)   
36                      Frï¿½ï¿½gil (feat. Grupo Front   
60                                              Tï¿½ï¿   
63                                                BESO   
..                                                 ...   
887                                    ALIEN SUPERSTAR   
915                                         Sin Seï¿½ï   
918                                      THE LONELIEST   
929                          Bamba (feat. Aitch & BIA)   
930                                Casei Com a Putaria   

                            artist(s)_name  artist_count  released_year  \
21                            Taylor Swift             1           2023   
26                     Rï¿½ï¿½ma, Selena G             2           2022   
36   Yahritza Y Su Esencia, Grupo Frontera       